### 1 - Loading package and configurations
- Setup openai configs
  - OPENAI_API_TYPE=xxxx
  - OPENAI_API_KEY=xxxxx
  - OPENAI_API_VERSION=xxxx
  - OPENAI_API_BASE=xxxxsbcSpoc
- setup langsmith 
  - LANGCHAIN_TRACING_V2=xxxx
  - LANGCHAIN_ENDPOINT=xxxxx
  - LANGCHAIN_API_KEY=xxxx
  - LANGCHAIN_PROJECT=xxxx
 
#### All the chat related configuration will be send to langsmithconfigs

In [1]:
import os
from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

# load env
load_dotenv()
# print out - test
print(os.getenv('OPENAI_API_TYPE'))
# create llm
llm = AzureChatOpenAI(deployment_name='chatGPTAzure', model='gpt-35-turbo')

azure


In [2]:
# create prompt template
prompt = ChatPromptTemplate.from_template("Show me description for company {company_name} in 100 words.")

# chain model and prompt
chain = prompt | llm | StrOutputParser()

chain

RunnableSequence(first=ChatPromptTemplate(input_variables=['company_name'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['company_name'], output_parser=None, partial_variables={}, template='Show me description for company {company_name} in 100 words.', template_format='f-string', validate_template=True), additional_kwargs={})]), middle=[AzureChatOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-35-turbo', temperature=0.7, model_kwargs={}, openai_api_key='d2d65893138c46d39ffe1d69f52eda88', openai_api_base='https://qucy-openai-test.openai.azure.com/', openai_organization='', openai_proxy='', request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None, tiktoken_model_name=None, deployment_name='chatGPTAzure', model_version='', openai_api_type='azure', openai_api_version

In [3]:
chain.invoke({"company_name": "HSBC"})

'HSBC Holdings plc is a British multinational banking and financial services company headquartered in London, England. It is one of the largest banks in the world, serving over 40 million customers in 64 countries and territories. The company offers a wide range of services including personal and commercial banking, wealth management, insurance, and investment banking. HSBC has a strong presence in Asia, Europe, and North America, and is committed to sustainable finance and environmental, social, and governance (ESG) practices. With a long history dating back to the mid-19th century, HSBC is a trusted and reliable partner for individuals, businesses, and institutions around the world.'

In [4]:
chain.invoke({"company_name": "Tesla"})

"Tesla, Inc. is an American electric vehicle and clean energy company founded by entrepreneur Elon Musk in 2003. The company's mission is to accelerate the world's transition to sustainable energy through the development, manufacturing, and sale of electric vehicles, solar panels, and energy storage systems. Tesla is known for its innovative technology, sleek designs, and commitment to sustainability. Its electric vehicles have set new standards for performance and efficiency, and its solar and energy storage solutions are helping to power homes and businesses around the world. Tesla has a global presence with operations in North America, Europe, and Asia."

### Let's try a agent

In [5]:
from langchain.utilities import SerpAPIWrapper
from langchain.llms.openai import OpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

search = SerpAPIWrapper(serpapi_api_key='dbee9f37664f6c7629d971c38f60417bd4b67e3e605e38920b20f219f6dc06f8')

tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)
self_ask_with_search.run(
    "What is the hometown of the reigning men's U.S. Open champion?"
)



> Entering new AgentExecutor chain...
I need to search for information related to the U.S. Open to find the answer to this question.
Action: Intermediate Answer
Action Input: "U.S. Open champion hometown"
Observation: Coco Gauff is the toast of tennis — and Delray Beach, Florida! Over the weekend, the US Open champion celebrated her historic Grand Slam win with family and friends in her hometown. “Me n only SOME of my favorite people 💖” the 19-year-old phenom wrote on Instagram alongside a carousel of happy photos.
Thought:This article is about Coco Gauff, who won the women's U.S. Open. I need to refine my search to find information specifically about the men's champion.
Action: Intermediate Answer
Action Input: "men's U.S. Open champion hometown"
Observation: ['Newport (1881–1914), Forest Hills (1915–1920, 1924–1977), and Philadelphia (1921–1923) held the event before it settled in 1978 at the USTA National Tennis ...']
Thought:Oops, this search result is not helpful. I need to refi

"Daniil Medvedev's hometown is Moscow, Russia."

### Embedding

In [6]:
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()

embeddings = embeddings_model.embed_documents(
    [
        "Hi there!",
        "Oh, hello!",
        "What's your name?",
        "My friends call me World",
        "Hello World!"
    ]
)
len(embeddings), len(embeddings[0])

(5, 1536)

In [7]:
## Test run on evaluation dataset

import langsmith
from langchain import chat_models, prompts, smith
from langchain.schema import output_parser

# Define your runnable or chain below.
prompt = prompts.ChatPromptTemplate.from_messages(
  [
    ("system", "You are a helpful AI assistant, that help people to answer different questions"),
    ("human", "{input}")
  ]
)
llm = chat_models.AzureChatOpenAI(deployment_name='chatGPTAzure', model='gpt-35-turbo')
chain = prompt | llm | output_parser.StrOutputParser()

# Define the evaluators to apply
eval_config = smith.RunEvalConfig(
    evaluators=[
        "cot_qa",
        smith.RunEvalConfig.LabeledCriteria("conciseness"),
        smith.RunEvalConfig.LabeledCriteria("relevance")
    ],
    custom_evaluators=[],
    eval_llm=chat_models.AzureChatOpenAI(deployment_name='chatGPTAzure', model='gpt-35-turbo')
)

client = langsmith.Client()
chain_results = client.run_on_dataset(
    dataset_name="wealth_rm_dataset",
    llm_or_chain_factory=chain,
    evaluation=eval_config,
    project_name="wealth_rm_dataset_test3",
    #concurrency_level=5,
    verbose=True,
)


KeyboardInterrupt

